In [1]:
pip install ollama


Note: you may need to restart the kernel to use updated packages.


# Load input file

In [2]:
project_title_map = {
        "archives_space": "Archives Space Project",
        "bad_camp": "BADCamp Project",
        "neurohub": "NeuroHub Project",
        "open_spending": "Open Spending Project",
        "planning_poker": "Planning Poker Project",
        "recycling": "Recycling Project"
    }
projects = ["archives_space", "bad_camp", "neurohub", "open_spending", "planning_poker", "recycling"]

selected_project = projects[0] # select which project you want to test with

project_path = "./input_files_temp/user_stories_{}/user_stories_{}_2.json".format(selected_project, selected_project)
with open(project_path, 'r') as file:
        project_content = file.read()
        project_title = project_title_map[selected_project]

# Load model and configure model parameters

In [3]:
import ollama
from ollama import Client

model_version = "llama3.1:8b-instruct-fp16"
#model_version = "llama3.1:70b"
num_ctx = 20480 # context length is higher because of the refinement process
temperature = 0 # should be kept 0 for deterministic results, default value 0.8

options = {"num_ctx": num_ctx, "temperature": temperature}
#model_name = "emir_" + model_version
model_name = model_version
#ollama.create(model=model_name, modelfile=modelfile)

one_shot = True

# Load one-shot example if set True

In [4]:
import json
if one_shot:
    with open('example_reasoning_1.txt', 'r') as file:
        example_run_text = file.read()
        example_run_prompt = f"""
        I will give you an example run for another software project just to show you the reasoning process and the output format. 
        VERY IMPORTANT NOTE: "NEVER COPY THE REASONING GIVEN IN THE EXAMPLE RUN! This Example is for Reference Only, Come Up With Your Own Reasoning for The User Input!"
        - EXAMPLE RUN START - 

        {example_run_text}
        
        - EXAMPLE RUN END -

        """
else:
    example_run_prompt = ""
example_run_prompt


'\n        I will give you an example run for another software project just to show you the reasoning process and the output format. \n        VERY IMPORTANT NOTE: "NEVER COPY THE REASONING GIVEN IN THE EXAMPLE RUN! This Example is for Reference Only, Come Up With Your Own Reasoning for The User Input!"\n        - EXAMPLE RUN START - \n\n        -INPUT-\n\nProject Title: Scrum Alliance Project\n\nCategorized User Stories:\n\n{\n    "description": "These stories were written to describe the functionality of an early version of the Scrum Alliance website, which is a platform dedicated to supporting and promoting the use of Scrum, a popular agile framework for managing and completing complex projects.",\n    "Usability & Accessibility": [\n      "As a site member, I want to describe myself on my own page in a semi-structured way, so that others can learn about me.",\n      "As a Practitioner, I want to include additional details about me in my profile page, so that I can showcase my exper

# Get first assessment from model

In [5]:
client = Client(
  headers={}
)
messages = []


generator_system_message = {'role': 'system', 'content': f"""
You are a software architect. Your job is to get a list of categorized user stories with a description, analyze them in detail and
assign a score for each architecture pattern depending on their relevance to the project and 
if it would prove useful in the implementation of the described project.
- The decision-making process should be independent of the order of the user stories and must remain consistent even if the user stories contain minor typos or formatting errors.

When the user gives you a list of user stories with a description, analyze the given user stories, give the reasoning for each architecture pattern, and then print out the assigned scores.
Be as objective as possible during your scoring. The decisions need to be deterministic and reproducible, regardless of the order of elements presented or any random factors.

Here are the architecture patterns you will score:
*Layered Architecture 
*Event-Driven Architecture
*Microkernel Architecture 
*Microservices Architecture 
*Space-Based Architecture
*Pipeline Architecture
*Client-Server Architecture

Here are the score options and their corresponding meaning:
1: "Completely unsuitable"
2: "Partially suitable"
3: "Moderately applicable"
4: "Well suited"
5: "Perfectly aligned"

IMPORTANT: The final scores must be given in json format after the detailed reasoning for each architecture pattern:

{{
  "Layered Architecture": <int>,
  "Event-Driven Architecture": <int>,
  "Microkernel Architecture": <int>,
  "Microservices Architecture":<int>,
  "Space-Based Architecture": <int>,
  "Pipeline Architecture": <int>,
  "Client-Server Architecture": <int>
}}

{example_run_prompt}

"""}
messages.append(generator_system_message)


messages.append({'role': 'user', 'content': f"""
I will give you a list of categorized user stories and a description created for a software project titled {project_title}. Please analyze them in detail and give me the scoring for each architecture pattern.
The final scores must be given in json format after the detailed reasoning for each architecture pattern:

{{
  "Layered Architecture": <int>,
  "Event-Driven Architecture": <int>,
  "Microkernel Architecture": <int>,
  "Microservices Architecture":<int>,
  "Space-Based Architecture": <int>,
  "Pipeline Architecture": <int>,
  "Client-Server Architecture": <int>
}}

Project Title: {project_title}

Categorized User Stories:

{project_content}

"""})
response = client.chat(model=model_name, messages=messages, options=options)
message = response['message']
print(message['content'])
messages.append(message)


Reasoning:

Layered Architecture: The Planning Poker Project involves distinct responsibilities such as presentation (handling user interactions, displaying information), business logic (managing games, estimators, and estimates), and data persistence (storing game data, estimator information). A layered architecture naturally separates these concerns, improving maintainability, testability, and the potential for independent scaling of layers in the future.

Event-Driven Architecture (Pub-Sub Architecture): There are some events such as importing stories from a spreadsheet or exporting transcripts of games. However, they don't appear to be the primary drivers of the system's functionality. A full-fledged pub-sub architecture might be overkill. However, using events for specific actions like notifications could be beneficial.

Microkernel Architecture: This pattern is not a good fit. The core functionality is relatively well-defined (game management, estimator management, estimate proce

# Self-refinement Process

In [6]:

# Let's capture the model's initial answer in a variable:
current_answer = message['content']  # e.g. the text with reasoning + final scores
evaluator_context = messages.copy() # this context will be used in evaluator messages
eval_system_message = {
        "role": "system",
        "content": """Your task is changed. You are now the Evaluator. 
Your job: By taking the previously given user stories into account, evaluate the given reasoning and scores for each architecture pattern.

Here are the score options and their corresponding meaning:
1: "Completely unsuitable"
2: "Partially suitable"
3: "Moderately applicable"
4: "Well suited"
5: "Perfectly aligned"

- Do not assume any new information about the project. The description and the user stories for the project remain the same.
- Only suggest a refinement for an assigned score when you are sure its score is wrong and it must change. IMPORTANT: If the current score is already reasonable DO NOT make a refinement suggestion.
- Also, go through your previous evaluations to ensure you don’t repeat the same refinement suggestion for an architecture pattern more than once for the same reason.

After your refinement suggestions, on a NEW line, end your response with exactly one of the two markers:
- REFINE (if refinements are needed)
- NO_REFINEMENT (if no refinements are needed)

Do not include any additional text after that marker.
Do not wrap it in quotes.

Important:
- Score can only take the integer values that are given as the score options, no float values.
- If you provide refinement for any of the pattern scores, do not use NO_REFINEMENT.
Because this will stop the whole process and the current scores will be left unrefined.
- Use NO_REFINEMENT only if you have zero refinements to suggest.
"""
}
evaluator_context.append(eval_system_message)

refiner_context = messages.copy() # this context will be used in refiner messages

refine_system_message = {
        "role": "system",
        "content": """Your task is changed. You are now the Refiner. 
You will be given the latest reasoning/scores for the project and the evaluator's feedback for the latest reasoning/scores.
Make changes on the current scores according to the given refinement.
IMPORTANT:  ONLY make a change for a score when the refinement suggestion for that score is reasonable and the assigned score must change. Otherwise keep it unchanged.

When giving your output refined scores, keep the same format in the updated response (reasoning and then the updated scores in json format).
Be as objective as possible during your refinement. The decisions need to be deterministic and reproducible, regardless of the order of elements presented or any random factors.

Here are the score options and their corresponding meaning:
1: "Completely unsuitable"
2: "Partially suitable"
3: "Moderately applicable"
4: "Well suited"
5: "Perfectly aligned"

- Do not assume any new information about the project. The description and the user stories for the project remain the same.
"""
    }
refiner_context.append(refine_system_message)
# We'll define how many refinement loops to allow:
MAX_REFINEMENT_ITERATIONS = 2

refinement_iterations = MAX_REFINEMENT_ITERATIONS # if not set again in the loop, it means the model went through max iterations
for i in range(MAX_REFINEMENT_ITERATIONS):
    # 1) Evaluate the current answer
    
    evaluate_prompt = {
        "role": "user",
        "content": f"""
            Evaluate the current reasoning and scores by taking the original categorized user stories into account and make refinement suggestions for the current answer:

            Current Answer:
            {current_answer}

            Refinement Suggestions:
            """
    }
    evaluator_context.append(evaluate_prompt)
    eval_response = client.chat(model=model_name, messages=evaluator_context, options=options)
    eval_message = eval_response['message']
    eval_feedback = eval_message['content'].strip()
    evaluator_context.append(eval_message)
    
    print(f"\n=== Evaluator Feedback (Iteration {i+1}) ===\n{eval_feedback}\n")
    
    # 2) If the evaluator indicates no changes are needed, break out
    if "NO_REFINEMENT" in eval_feedback:
        print("Evaluator says no changes are needed. Stopping refinement.")
        refinement_iterations = i
        break
    
    # 3) Otherwise, refine
    refine_prompt = {
        "role": "user",
        "content": f"""
            Analyze the previous answer and the feedback of the evaluator, and then refine your previous answer by taking the feedback of the evaluator into account. 
            Keep the same format (reasoning and then the final scores in json format).

            Previous Answer:
            {current_answer}

            Evaluator Feedback:
            {eval_feedback}

            Revised Answer:
            """
    }
    refiner_context.append(refine_prompt)
    
    refine_response = client.chat(model=model_name, messages=refiner_context, options=options)
    refine_message = refine_response['message']
    revised_answer = refine_message['content'].strip()
    refiner_context.append(refine_message)
    
    print(f"=== Refined Answer (Iteration {i+1}) ===\n{revised_answer}\n")
    
    # Update current_answer for potential next iteration
    current_answer = revised_answer

# After the loop, current_answer holds the final refined output:
print("=== Final Refined Scores & Reasoning ===\n", current_answer)


=== Evaluator Feedback (Iteration 1) ===
Refinement Suggestions:

* Layered Architecture: The current score of 4 seems reasonable, but upon closer inspection, some user stories suggest a need for more complex business logic, such as importing and exporting data (e.g., "As a moderator, I want to import stories from a spreadsheet, so that I don't have to copy and paste each individual story."). This might warrant a score of 5, indicating that the layered architecture is well suited for the project's requirements.
* Event-Driven Architecture: The current score of 2 seems too low. While events are not the primary drivers of the system's functionality, there are several user stories that involve notifications (e.g., "As a Researcher, I want results to be stored in a non-identifiable way, so that I can study the data..."). This suggests that an event-driven architecture could be beneficial for handling these notifications. A score of 3 or 4 might be more appropriate.
* Microkernel Architect

# Summarize the evaluation result

In [7]:
refiner_context.append({'role': 'system', 'content': """
Ok now the evaluation process is finished. take the first assessment and the last assessment from the user. 
And just return the final scores before and after the evaluation-refinement process
following this json format below:

{
  "Layered Architecture": {"before": <int>, "after": <int>},
  "Event-Driven Architecture": {"before": <int>, "after": <int>},
  "Microkernel Architecture": {"before": <int>, "after": <int>},
  "Microservices Architecture": {"before": <int>, "after": <int>},
  "Space-Based Architecture": {"before": <int>, "after": <int>},
  "Pipeline Architecture": {"before": <int>, "after": <int>},
  "Client-Server Architecture": {"before": <int>, "after": <int>}
}
"""})

refiner_context.append({'role': 'user', 'content': f"""
read the first assessment and the last assessment, and return the final scores before and after the evaluation-refinement process
following this json format below:

{{
  "Layered Architecture": {{"before": <int>, "after": <int>}},
  "Event-Driven Architecture": {{"before": <int>, "after": <int>}},
  "Microkernel Architecture": {{"before": <int>, "after": <int>}},
  "Microservices Architecture": {{"before": <int>, "after": <int>}},
  "Space-Based Architecture": {{"before": <int>, "after": <int>}},
  "Pipeline Architecture": {{"before": <int>, "after": <int>}},
  "Client-Server Architecture": {{"before": <int>, "after": <int>}}
}}


first assessment:
{refiner_context[2]["content"].strip()}

last assessment:
{current_answer.strip()}
"""})
response = client.chat(model=model_name, messages=refiner_context, options={"num_ctx": num_ctx, "temperature": 0}) # higher temperatures are too varied
message = response['message']
refiner_context.append(message)
print(message["content"])


Here are the final scores before and after the evaluation-refinement process:

{
  "Layered Architecture": {"before": 4, "after": 6},
  "Event-Driven Architecture": {"before": 2, "after": 4},
  "Microkernel Architecture": {"before": 1, "after": 3},
  "Microservices Architecture": {"before": 4, "after": 7},
  "Space-Based Architecture": {"before": 1, "after": 3},
  "Pipeline Architecture": {"before": 2, "after": 5},
  "Client-Server Architecture": {"before": 5, "after": 5}
}


# Print log to output file

In [8]:
from datetime import datetime
import json
from pathlib import Path
Path("./logs/self-refinement-one-shot").mkdir(parents=True, exist_ok=True)
Path("./logs/self-refinement-zero-shot").mkdir(parents=True, exist_ok=True)

final_data = {
        "modelName": model_name,
        "temperature": temperature,
        "context_limit": num_ctx,
        "projectTitle": project_title,
        "selfRefinement": True,
        "oneShot": one_shot,
        "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
        "lastMessage": refiner_context[-1]["content"],
        "numberOfIterations": refinement_iterations,
        "messages": [message["content"] for message in refiner_context],
    }
    
# 3) Generate a filename based on model name and current timestamp
if one_shot:
    filename = f"./logs/self-refinement-one-shot/log_{model_version}_{final_data['timestamp']}.json"
else:
    filename = f"./logs/self-refinement-zero-shot/log_{model_version}_{final_data['timestamp']}.json"
# 4) Write the conversation to a JSON file
with open(filename, "w", encoding="utf-8") as f:
    json.dump(final_data, f, indent=2, ensure_ascii=False)

print(f"Assesment complete. The whole conversation is saved to {filename}")

Assesment complete. The whole conversation is saved to ./logs/self-refinement-one-shot/log_llama3.1:8b-instruct-fp16_20250202_010603.json
